## Sztuczne sieci neuronowe - laboratorium 8

### Splotowe sieci neuronowe - cz. 2

Na poprzednich zajęciach poznaliśmy warstwy tworzące **splotową sieć neuronową** i nauczyliśmy się tworzyć modele jako klasy  dziedziczące po `nn.Module`.

Dziś wytrenujemy splotową sieć neuronową do binarnej klasyfikacji obrazu i poznamy dodatkowe techniki stosowane w sieciach neuronowych, m.in. do regularyzacji modeli.

#### Pytania kontrolne

1. Opisz budowę splotowej sieci neuronowej. Wyjaśnij, do czego służą jej poszczególne warstwy.
2. Na czym polega regularyzacja modeli?
3. Jakie znasz metody regularyzacji stosowane w sieciach neuronowych?

### Z poprzednich ćwiczeń

Uruchom kolejne komórki, wykorzystujące kod z poprzednich zajęć, aby przygotować zbiór danych - `cifar2` oraz klasę `Net` definiującą model.

In [69]:
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [70]:
class_names = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}

In [71]:
tensor_cifar10 = datasets.CIFAR10("data", train=True, download=False, transform=transforms.ToTensor())
tensor_cifar10_val = datasets.CIFAR10("data", train=False, download=False, transform=transforms.ToTensor())

In [72]:
imgs = torch.stack([img_t for img_t, _ in tensor_cifar10], dim=3)
per_channel_means = imgs.view(3, -1).mean(dim=1)
per_channel_std = imgs.view(3, -1).std(dim=1)

In [73]:
transforms_compose = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(per_channel_means, per_channel_std)
])

transformed_cifar10 = datasets.CIFAR10("data", train=True, download=False, transform=transforms_compose)
transformed_cifar10_val = datasets.CIFAR10("data", train=False, download=False, transform=transforms_compose)

In [74]:
label_map = {0: 0, 2: 1}
new_class_names  = [class_names[i] for i in label_map]

cifar2 = [(img, label_map[label]) for img, label in tensor_cifar10 if label in label_map]
cifar2_val = [(img, label_map[label]) for img, label in tensor_cifar10 if label in label_map]

In [75]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

### Trening modelu

#### Ćwiczenie
Uzupełnij poniższe komórki, aby wytrenować splotową sieć neuronową do zadania klasyfikacji binarnej.

Przyjmij learning rate o wartości 0.01 i batch size 64. Trenuj przez 100 epok. Użyj optymaliatora SGD.

In [76]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    model.train()
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to()
            labels = labels.to()
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
        loss_train /= len(train_loader.dataset)

        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train}")

In [77]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)

model = Net()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader
)

Epoch 1, Training loss 0.010727701449394226
Epoch 10, Training loss 0.007026612052321434
Epoch 20, Training loss 0.0056229409098625185
Epoch 30, Training loss 0.005117171007394791
Epoch 40, Training loss 0.004846780079975724
Epoch 50, Training loss 0.004626769314706325
Epoch 60, Training loss 0.004410410498082638
Epoch 70, Training loss 0.0042155707865953445
Epoch 80, Training loss 0.0040079746186733245
Epoch 90, Training loss 0.003815111853182316
Epoch 100, Training loss 0.00363273723423481


### Walidacja modelu

#### Ćwiczenie

Zaimplementuj funkcję `validate`, która zmierzy dokładność wytrenowanego modelu na dwóch zbiorach - uczącym i walidacyjnym.

Porównaj wyniki z wynikami z laboratorium nr 5 (sieć gęsta).

In [78]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)

In [79]:
def validate(model, train_loader, val_loader):
    model.eval()
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to()
                labels = labels.to()
                outputs = model(imgs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total

        print(f"{name} accuracy: {accuracy:.2f}")

In [80]:
validate(model, train_loader, val_loader)

train accuracy: 0.90
val accuracy: 0.90


### Zapis i odczyt modelu

Wytrenowany model (zwłaszcza tak, którego trening trwa długo) warto zapisać, aby móc go użyć później.

Typowo po każdej epoce treningu sprawdza się działanie modelu na zbiorze walidacyjnym (walidacja).
Zapisu modelu (tzw. "checkpoint") typowo dokonuje się, jeśli wartość danej metryki (np. dokładność lub F1 na zbiorze walidacyjnym) jest lepsza niż najlepsza uzyskana dotychczas.

PyTorch pozwala zapisać wagi (parametry) modelu z użyciem `torch.save` oraz tzw. `state_dict` modelu (https://pytorch.org/tutorials/recipes/recipes/what_is_state_dict.html). Innym sposobem zapisu jest zapis całego modelu (z użyciem `pickle` "pod spodem"):
https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_models_for_inference.html

Następnie, do wczytania zapisanego modelu można użyć metody `load_state_dict` (oraz `torch.load`), jeśli zapisywaliśmy tylko `state_dict` lub tylko `torch.load`, jeśli zapisywaliśmy cały model.

In [81]:
print(model.state_dict())

OrderedDict({'conv1.weight': tensor([[[[ 2.6952e-02,  1.5811e-01,  2.5854e-01],
          [ 1.7185e-01,  2.1432e-01,  1.4635e-01],
          [ 9.8063e-02,  4.8224e-02,  3.1590e-01]],

         [[ 5.5910e-03, -7.7587e-02, -1.5105e-01],
          [-1.8848e-01,  9.7199e-03, -2.9531e-01],
          [-4.5475e-02,  2.2443e-03, -1.0404e-01]],

         [[-8.4342e-02, -1.9070e-01, -2.8194e-01],
          [-1.8346e-01, -6.2627e-02, -3.1266e-01],
          [-1.5285e-01, -3.1271e-01, -2.0209e-01]]],


        [[[ 2.8254e-02, -6.3556e-02,  2.5772e-01],
          [ 2.9880e-01, -2.0441e-01,  1.2164e-01],
          [-1.2513e-01,  5.1281e-02,  1.5160e-01]],

         [[-2.2792e-01, -5.6854e-02,  9.1111e-02],
          [ 1.6413e-01, -2.7808e-01,  6.1409e-03],
          [-1.4743e-01,  5.3272e-02, -6.5304e-02]],

         [[-1.2947e-01, -1.1153e-01,  2.0584e-01],
          [-1.7568e-01, -3.8679e-01,  9.4861e-02],
          [-1.0282e-02,  1.3125e-01, -5.7939e-02]]],


        [[[-2.1533e-01,  1.7195e-01, 

In [82]:
torch.save(model.state_dict(), "data/birds_vs_airplanes.pt")

In [83]:
loaded_model = Net()
loaded_model.load_state_dict(torch.load("data/birds_vs_airplanes.pt"))

<All keys matched successfully>

### Trening na GPU (opcjonalnie)

Aby przyspieszyć trening (zwłaszcza w przypadku głębokich modeli i dużych zbiorów danych), powszechnie stosuje się karty graficzne (GPU). Jeśli mamy dostęp do maszyny z kartą graficzną (najlepiej od NVIDIA, obsługującą CUDA), możemy łatwo "przenieść" trening na GPU.

W tym celu należy przenieść zarówno dane, jak i model, na kartę graficzną, używając metody `.to` (tensora i `nn.Module`) na zdefiniowane urządzenie (patrz poniżej).

In [84]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Training on {device}")

Training on cuda


#### Ćwiczenie

Zmodyfikuj napisaną wyżej pętlę treningową oraz inicjalizację modelu, przenosząc odpowiednio dane (obrazki i etykiety) oraz model na `device`.

In [85]:
def training_loop_gpu(n_epochs, optimizer, model, loss_fn, train_loader):
    model.train()
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train}")

In [86]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)

model = Net().to(device=device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()


training_loop_gpu(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader
)

Epoch 1, Training loss 107.0023803114891
Epoch 10, Training loss 66.44348514080048
Epoch 20, Training loss 54.032825261354446
Epoch 30, Training loss 50.942595556378365
Epoch 40, Training loss 48.321190878748894
Epoch 50, Training loss 46.118479162454605
Epoch 60, Training loss 43.678450144827366
Epoch 70, Training loss 41.2571586817503
Epoch 80, Training loss 39.37156615406275
Epoch 90, Training loss 37.459661327302456
Epoch 100, Training loss 35.56719648092985


### Rozbudowa modelu

Możemy "powiększyć" model "na szerokość" (dodać więcej filtrów) lub "na głębokość" (dodać więcej warstw).

#### Ćwiczenie

Zmodyfikuj klasę `Net` i stwórz kolejno:
- `NetWidth` - 2x więcej filtrów w warstwach splotowych (niech liczba filtrów będzie argumentem konstruktora)
- `NetDepth` - dodatkowa warstwa splotowa `conv3`

Pamiętaj o zmodyfikowaniu metody `forward`.

In [87]:
class NetWidth(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 * 2, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(n_chans1 * 2 * 8 * 8, n_chans1 * 4)
        self.fc2 = nn.Linear(n_chans1 * 4, 2)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, self.n_chans1 * 2 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [88]:
class NetDepth(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(n_chans1, n_chans1, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(n_chans1 * 4 * 4, n_chans1 * 4)
        self.fc2 = nn.Linear(n_chans1 * 4, 2)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = torch.tanh(self.conv2(out))
        out = F.max_pool2d(torch.tanh(self.conv3(out)), 2)
        out = out.view(-1, self.n_chans1 * 4 * 4)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

### Regularyzacja L2 / weight decay

Regularyzację L2 można zaimplementować samemu (jak niżej).

Jest ona jednak wbudowana w `torch.optim`, np. https://pytorch.org/docs/stable/optim.html#torch.optim.SGD, gdzie wystarczy podać wartość `weight_decay` tworząc optymalizator.

#### Ćwiczenie

W poniższej pętli treningowej dopisz fragment realizujący regularyzację L2 dla lambda = 0.001.

In [89]:
def training_loop_l2reg(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            l2_reg = 0
            for param in model.parameters():
                l2_reg += torch.norm(param, 2)
            loss += 0.001 * l2_reg

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train}")

In [90]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)

model = Net().to(device=device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop_l2reg(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader
)

Epoch 1, Training loss 108.10475647449493
Epoch 10, Training loss 75.13450002670288
Epoch 20, Training loss 59.52363866567612
Epoch 30, Training loss 54.52742849290371
Epoch 40, Training loss 51.23729193210602
Epoch 50, Training loss 48.53098799288273
Epoch 60, Training loss 46.67361645400524
Epoch 70, Training loss 44.40358214080334
Epoch 80, Training loss 42.65197682380676
Epoch 90, Training loss 40.839143201708794
Epoch 100, Training loss 39.36301167309284


#### Ćwiczenie

Wywołaj "zwykłą" pętlę treningową, tym razem podająć `weight_decay` optyamlizatora równe 0.001. Zaobserwuj wpływ na funkcję straty.

In [91]:
training_loop_gpu(
    n_epochs = 100,
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=0.001),
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader
)

Epoch 1, Training loss 37.02860698103905
Epoch 10, Training loss 35.84481417387724
Epoch 20, Training loss 34.16949528455734
Epoch 30, Training loss 32.87816595286131
Epoch 40, Training loss 31.381581768393517
Epoch 50, Training loss 29.919028103351593
Epoch 60, Training loss 28.861051090061665
Epoch 70, Training loss 27.01045712083578
Epoch 80, Training loss 25.870101675391197
Epoch 90, Training loss 24.33152087032795
Epoch 100, Training loss 23.205924697220325


Funkcja straty przyjmuje niższe wartości.

### Dropout

Na poprzednich zajęciach poznaliśmy technikę regularyzacji modeli - tzw. "regularyzację L2" (a.k.a. "weight decay"). Polega ona na modyfikacji funkcji straty poprzez dodanie odpowiedniego cżłonu (lub modyfikacji kroku aktualizacji parametrów w algorytmie optymalizacyjnym), wpływając bezpośrednio na wartości wag (parametrów) modelu.

Innym z mechanizmów regularyzacyjnych typowym dla (głębokich) sieci neuronowych jest **dropout**:
- (Srivastava et al., "Dropout: A Simple Way to Prevent Neural Networks from Overfitting", 2014) https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf

Dropout polega na losowym wyłączaniu części neuronów w każdej iteracji treningu, co powoduje, że żadna porcja danych nie jest "widziana" przez całą sieć. Dla danej warstwy z dropoutem określa się prawdopodobieństwo wyłączenia (w PyTorch; lub pozostawienia - uwaga na różne konwencje!) danego neuronu.

- https://pytorch.org/docs/stable/generated/torch.nn.Dropout2d.html

**Uwaga**:  

Dropout jest przykładem mechanizmu, który zachowuje się inaczej w trakcie treningu i walidacji. Tworząc modele w PyTorch należy pamiętać, aby przełączyć je w odpowiedni tryb: `model.train()` lub `model.eval()`. Dobrze jest wyrobić sobie nawyk używania tych trybów nawet, jeśli w modelu nie ma dropoutu (ani np. batch normalization - patrz poniżej).

#### Ćwiczenie
Dołóż warstwy `nn.Dropout2d` po warstwach splotowych (po max poolingu) do sieci `Net` i stwórz w ten sposob `NetDropout`.

In [92]:
class NetDropout(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(n_chans1 * 8 * 8, n_chans1 * 4)
        self.fc2 = nn.Linear(n_chans1 * 4, 2)
        self.dropout = nn.Dropout2d(p=0.5)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = self.dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, self.n_chans1 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.dropout(out)
        out = self.fc2(out)
        return out

### Batch Normalization

Ważnym mechanizmem w kontekście (głębokich) sieci neuronowych jest tzw. **batch normalization**:

- (Ioffe i Szegedy, "Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift", 2015) - https://arxiv.org/pdf/1502.03167.pdf

Polega on na normalizacji (standaryzacji) wejść do funkcji aktywacji (lub wyjść z nich - w praktyce nie powinno mieć to większego znaczenia):
- w treningu: na podstawie średniej i wariancji pojedynczego wsadu danych ("batch")
- podczas inferencji: na podstawie średniej i wariancji całego zbioru uczącego (estymowane w czasie treningu)

Zastosowanie BN:
- pozwala ustabilizować / przyspieszyć trening
- zapobiega zanikaniu gradientów
- wprowadza dodatkowy efekt regularyzacyjny

W PyTorch dla sieci splotowych BN zrealizowana jest jako:
- https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html

Podobnie jak w przypadku dropoutu, należy przełączać model między trybem treningu i ewaluacji.

#### Ćwiczenie

Dodaj warstwy `BatchNorm2d` po warstwach splotowych `Net` tworząc w ten sposób `NetBatchNorm`.

In [93]:
class NetBatchNorm(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(n_chans1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(n_chans1)
        self.fc1 = nn.Linear(n_chans1 * 8 * 8, n_chans1 * 4)
        self.fc2 = nn.Linear(n_chans1 * 4, 2)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = torch.tanh(out)
        out = F.max_pool2d(out, 2)

        out = self.conv2(out)
        out = self.bn2(out)
        out = torch.tanh(out)
        out = F.max_pool2d(out, 2)

        out = out.view(-1, self.n_chans1 * 8 * 8)
        out = self.fc1(out)
        out = torch.tanh(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out

### Wnioski

W trakcie zajęć poznaliśmy kilka technik regularyzacji modeli, które można stosować w sieciach neuronowych:
- regularyzacja L2 (weight decay)
- dropout
- batch normalization.
Dzięki tym technikom możemy trenować głębsze i bardziej złożone modele, które są mniej podatne na przeuczenie.